# Classification widget for collision avoidance task

The widget generated by this notebook facilitates the creation of a dataset suitable for addressing a collision avoidance problem on Jetracer robot. [Jetbot robot code](https://github.com/NVIDIA-AI-IOT/jetbot) is used as a reference for creating the widget.

The approach we take to avoiding collisions is to create a virtual "safety bubble" around the robot. Within this safety bubble, the robot is able to spin in a circle without hitting any objects. Only a sensor is used: the camera mounted on Jetracer.

Two scenarios can be distinguished:
- When the robot's "safety bubble" is breached. This scenario will be labeled as `blocked`;
- When the robot's "safety bubble" is not breached. This scenario will be labeled as `free`.

> **The steering angle of the JetRacer is limited**. This is an important detail to remember when you're trying to estimate whether the robot's safety bubble is violated or not.

## Dataset

A folder dataset is created and it will contain two sub-folders `free` and `blocked`, where we'll place the images for each scenario. 

If you refresh the Jupyter file browser on the left, you should now see those directories appear. Next, let's create some buttons that we'll use to save snapshots for each class label. We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make sure we collect about as many free images as blocked images.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')
    
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

#display(widgets.HBox([free_count, free_button]))
#display(widgets.HBox([blocked_count, blocked_button]))

The camera is initialized.

In [ ]:
import traitlets 
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=65)
camera.running = True

image_widget = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

#display(image_widget)

To make sure we don't repeat any file names (even across different machines!) we'll use the `uuid` package in python, which defines the uuid1 method to generate a unique identifier. This unique identifier is generated from information like the current time and the machine address.

In [ ]:
from uuid import uuid1

count_free = 0
count_blocked = 0

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    cv2.imwrite(image_path, camera.value.copy())

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))

def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

## All together!

The complete widget is displayed. It presents real-time images from the camera, and the `free` and `blocked` buttons can be used to collect images of the two classes. The number of elements in each class is displayed next to each button.

In [ ]:
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))
display(image_widget)